In [ ]:
'''
安裝 tika

參考網頁
https://pypi.org/project/tika/
'''
!pip install tika

In [ ]:
'''
東吳大學 政治學報
https://web-ch.scu.edu.tw/politics/file/11106

參考連結:
[1] python selenium 對瀏覽器標籤頁進行關閉和切換
https://www.itread01.com/content/1543567328.html
'''


'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 正規表達式
import re

# 使用 tika 的 parser
from tika import parser

'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
yt-dlp 下載影音的好工具
參考網址：https://github.com/yt-dlp/yt-dlp
'''

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome( 
    options = my_options, 
    executable_path = driver_exec_path
)

# 放置爬取的資料
listData = []

# 建立資料夾
folderPath = 'parsed_files'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 網址
url = 'https://web-ch.scu.edu.tw/politics/file/11106'

'''
函式
'''
# 走訪頁面
def visit():
    driver.get(url);
    
# 取得主要連結
def getMainData():
    try:
        # 等待主要連結出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (
                    By.CSS_SELECTOR, 
                    "div#rndbox_body table.table.table-striped.table-bordered tbody td a"
                )
            )
        )
        
        # 得到所有連結的數量
        count = len(driver.find_elements(By.CSS_SELECTOR, "div#rndbox_body table.table.table-striped.table-bordered tbody td a"))
        
        # 切換到新分頁(初始分頁代號為 0，新開的為 1，所以切換到 1，代表移到分頁去操作)
        for i in range(count):
            # 開啟新分頁
            driver.execute_script(f'window.open("{url}", "_blank");')
            
            # 等待一下
            sleep(7)
            
            # 切換到分頁
            driver.switch_to.window(driver.window_handles[1])
            
            # 等元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (
                        By.CSS_SELECTOR, 
                        "div#rndbox_body table.table.table-striped.table-bordered tbody td a")
                )
            )
            
            # 取得 a 元素集合
            a_elms = driver.find_elements(By.CSS_SELECTOR, "div#rndbox_body table.table.table-striped.table-bordered tbody td a")
            
            # 按下超連結
            a_elms[i].click()
        
            # 等 pdf 資訊出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (
                        By.CSS_SELECTOR,
                        "div#article ol li h2 a"
                    )
                )
            )
            
            # 輸出所有 pdf 資訊
            a_article_elms = driver.find_elements(By.CSS_SELECTOR, "div#article ol li h2 a")
            for a_article in a_article_elms:
                print(f"文章名稱: {a_article.get_attribute('innerText')}")
                print(f"PDF連結: {a_article.get_attribute('href')}")
                
                # 簡單資料清理
                title = re.sub(r"\?|’|:| |\s", "_", a_article.get_attribute('innerText'))
                
                # 讀取兩篇 pdf 的內文，並寫入檔案
                with open(f"{folderPath}/{title}.txt", "w", encoding="utf-8") as file:
                    parsed_pdf = parser.from_file( a_article.get_attribute('href') )
                    file.write( parsed_pdf['content'] )
            
            # 關閉當前分頁
            driver.close()
            
            # 切換到初始分頁
            driver.switch_to.window(driver.window_handles[0])
            
    except TimeoutException:
        print("等不到指定元素出現…")

        
# 關閉瀏覽器
def close():
    driver.quit()
        
'''
主程式
'''
if __name__ == '__main__':
    visit()
    getMainData()
    close()